In [484]:
import re
import struct
import os
from collections import defaultdict
from tqdm.notebook import tqdm
import codecs
import gzip
from copy import deepcopy
import operator

#### Начнем с прямого индекса

In [485]:
class Fibonnachi:
    
    def __init__(self):
        self.fibonnachi_list = [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610,
                                987, 1597, 2584, 4181, 6765, 10946, 17711, 28657, 46368, 75025, 
                                121393, 196418, 317811, 514229, 832040]
    
    def encode(self, arr):
        value_list = ''
        flag = False
        for value in arr:
            value_list += '1'
            for k in self.fibonnachi_list[2:][::-1]:
                if k <= value:
                    value_list += '1'
                    flag = True
                    value -= k
                elif flag:
                    value_list += '0'
            flag = False
        return value_list
    
    def decode_elem(self, elem):
        value = 0
        for i in range(len(elem)):
            if elem[::-1][i] == '1':
                value += self.fibonnachi_list[2:][i]
        return value
    
    def decode(self, value_str):
        value_str = value_str[::-1]
        flag = False
        value_arr = []
        string=''
        for i in range(len(value_str)):
            if value_str[i] == '1':
                if flag:
                    value_arr.append(self.decode_elem(string))
                    flag = False
                    string = ''
                else:
                    flag = True
                    string = '1' + string
            else:
                string = '0' + string
                flag = False

        return value_arr[::-1]

In [486]:
class Indexing:
    def __init__(self):
        self.forward_index = defaultdict(list) # list of words
        self.backward_index = defaultdict(list) # list of doc_id
        self.doc_url = defaultdict(str)
        
        
    def build_forward_index(self, path_to_dataset):
        '''
        Построение прямого индекса по документам
        '''
        file_list = os.listdir(path_to_dataset)
        doc_id = 1
        for i in file_list:
            with gzip.open(path_to_dataset + '/' + i, 'rb') as f:
                while True:
                    first_byte = f.read(4)
                    if first_byte == b'':
                        break
                    size = struct.unpack('i', first_byte)[0]
                    text = f.read(size).decode('utf-8', 'ignore')
                    txt = re.split(r'\x1a{1,}', text)
                    url = txt[0]
                    self.doc_url[doc_id] = url[url.find('http'):]
                    tmp = []
                    for lines in txt[1:]:
                        tmp = re.findall(r'\w+', lines.lower())
                    self.forward_index[doc_id] = tmp
                    # периодически встречаются слова с дефисами... но решил их выкинуть)) под \w+ не подходят ведь)
                    doc_id += 1
                

                
    def build_backward_index(self):
        '''
        Построение обратного индекса. Просто берем слова без повторов из прямого индекса
        '''
        for key in self.forward_index.keys():
            for word in set(self.forward_index[key]):
                self.backward_index[word].append(key)
                
        self.backward_index_dict = deepcopy(self.backward_index)
                
                
    def forward_index_optimization(self, coder=Fibonnachi()):
        '''
        Оптимизируем прямой индекс. Каждому doc_id приписываем текст без повторяющихся в нем слов
        и кодируем с помощью codecs.encode()
        '''
        self.coder = coder
        for key in self.forward_index.keys():
            set_values = set(self.forward_index[key])
            self.forward_index[key] = codecs.encode(' '.join(list(set_values)))
    
    def backward_index_optimization(self, coder=Fibonnachi()):
        '''
        Терм в обратном индексе заменяем на его word_id, кодируем. Записываем posting list из промежутков
        между документами, кодируем posting list кодом Фибонначи
        '''
        self.coder = coder
        encoded_backward_index = {}
        cnt = 1 # счетчик обычный
        for key in self.backward_index.keys():
            encoded_backward_index[self.coder.encode([cnt])] = self.backward_index[key]
            cnt += 1
        self.backward_index = deepcopy(encoded_backward_index)
        encoded_backward_index.clear()

        for key in self.backward_index.keys():
            posting_list = []
            posting_list.append(self.backward_index[key][0])
            for i in range(1, len(self.backward_index[key])):
                posting_list.append(self.backward_index[key][i] - self.backward_index[key][i-1])
            self.backward_index[key] = self.coder.encode(posting_list)
            
    def make_dict(self):
        '''
        Для быстрого поиска создаем словарь с word_id, который считает как просто порядковый номер слова 
        в обратном индексе
        '''
        self.dict = {}
        cnt = 1 # счетчик обычный
        for key in self.backward_index_dict.keys():
            self.dict[key] = cnt
            cnt += 1
        self.backward_index_dict.clear()      
            

In [487]:
class Query_tree:
    def __init__(self):
        self.tree = {}
        
    def build(self, tokens, pos):
        '''
        Рекурсивно разбираем запрос и строим дерево запроса
        '''
        cnt = 0
        res = []
        for i in range(len(tokens) - 1, -1, -1):
            if tokens[i] == ')':
                cnt += 1
            elif tokens[i] == '(':
                cnt -= 1  
            elif tokens[i] == '|':
                res.append((i, cnt))
        for i in range(len(tokens) - 1, -1, -1):
            if tokens[i] == ')':
                cnt += 1
            elif tokens[i] == '(':
                cnt -= 1  
            elif tokens[i] == '&':
                res.append((i, cnt))
        # ищем минимум по cnt
        if len(res) != 0:
            res.sort(key = operator.itemgetter(1))
            self.tree[pos] = tokens[res[0][0]]
            self.build(tokens[:res[0][0]], 2*pos + 1)
            self.build(tokens[res[0][0] + 1:], 2*pos + 2)
        else:
            for i in range(len(tokens)):
                if tokens[i] == '!':
                    self.tree[pos] = tokens[i]
                    self.build(tokens[i+1:], 2*pos + 1)
                    break
                elif ((tokens[i] != '(') and (tokens[i] != ')')):
                    self.tree[pos] = tokens[i]

        
    def build_tree(self, query):
        '''
        Разделяем запрос на токены. Запускаем рекурсию
        '''
        tokens = re.findall(r'\w+|[\(\)&\|!]', query)
        self.build(tokens, 0)
        
        
                    
    def build_posting_lists(self, index):
        
        for key in self.tree.keys():
            if ((self.tree[key] != '!') and (self.tree[key] != '|') 
                and (self.tree[key] != '&')):
                if self.tree[key] in index.dict.keys():

                    posting_list = index.coder.decode(index.backward_index[
                        index.coder.encode([index.dict[self.tree[key]]])])
                    
                    for i in range(1, len(posting_list)):
                        posting_list[i] = posting_list[i] + posting_list[i-1]   
                    self.tree[key] = posting_list
                else:
                    self.tree[key] = []
        
                         
    def evaluate(self, pos):
        if self.tree[pos] == '&':
            left = self.evaluate(2*pos + 1)
            right = self.evaluate(2*pos + 2)
            return max(left, right)
        elif self.tree[pos] == '|':
            left = self.evaluate(2*pos + 1)
            right = self.evaluate(2*pos + 2)
            return min(left, right)
        elif self.tree[pos] == '!':
            val = self.evaluate(2*pos + 1)
            if val != self.doc_id:
                return self.doc_id
            else:
                return self.max_doc + 1
        if len(self.tree[pos]) != 0:
            return self.tree[pos][0]
        else:
            return self.max_doc + 1
        
    def goto(self, doc_id):
        for key in self.tree.keys():
            if ((self.tree[key] != '!') and (self.tree[key] != '|') 
                and (self.tree[key] != '&')):
                i = 0
                while ((i < len(self.tree[key])) and (self.tree[key][i] < doc_id)):
                    i += 1
                self.tree[key] = self.tree[key][i:]
    
    def return_urls(self, index_url):
        '''
        Потоковая обработка дерева
        '''
        self.url_id = []
        self.doc_id = 1
        self.max_doc = len(index.forward_index.keys())

        while self.doc_id <= self.max_doc:
            self.goto(self.doc_id)
            if self.doc_id == self.evaluate(0):
                self.url_id.append(self.doc_id)
            self.doc_id += 1

        urls = []

        for i in sorted(self.url_id):
            urls.append(index_url[i])
        
        return urls  

In [488]:
index = Indexing()

In [489]:
index.build_forward_index("./dataset")

In [490]:
index.build_backward_index()

In [491]:
index.forward_index_optimization()

In [492]:
index.backward_index_optimization()

####  Построение словаря

In [493]:
index.make_dict()

#### С индексом разобрались,  смотрим на дерево запросов

In [501]:
query_tree = Query_tree()

In [502]:
query = input().lower()

власти & США


In [503]:
query_tree.build_tree(query)

In [504]:
query_tree.tree

{0: '&', 1: 'власти', 2: 'сша'}

In [505]:
query_tree.build_posting_lists(index)

In [506]:
res = query_tree.return_urls(index.doc_url)

In [507]:
print(len(res))

294


In [509]:
for url in res:
    print(url)

http://lenta.ru/news/2014/04/23/usarmy/
http://lenta.ru/articles/2013/05/20/china/
http://lenta.ru/news/2014/03/10/nprnewyork/
http://lenta.ru/articles/2014/05/14/sanctions
http://lenta.ru/news/2012/02/07/homs/
http://lenta.ru/mideast/2002/07/15/syria
http://lenta.ru/russia/2001/04/05/parking/
http://lenta.ru/news/2016/01/13/no_bomb_please/
http://lenta.ru/news/2013/01/03/adoption/
http://lenta.ru/world/2002/07/11/jordan/
http://lenta.ru/news/2011/08/02/tarbaby/
http://lenta.ru/lib/14160961/full
http://lenta.ru/world/2000/03/18/patriq/
http://lenta.ru/mideast/2003/02/05/ramon/
http://lenta.ru/articles/2010/01/13/gasoil/
http://lenta.ru/news/2011/08/11/brodskij/
http://lenta.ru/news/2014/12/12/cuba/
http://lenta.ru/news/2011/04/19/mansion1/
http://lenta.ru/world/2004/11/05/blair/
http://lenta.ru/iraq/2003/07/17/guard/
http://lenta.ru/articles/2013/04/08/latvia/
http://lenta.ru/articles/2006/09/11/plot/
http://lenta.ru/articles/2008/12/19/iraq/
http://lenta.ru/articles/2004/05/19/abhazia